In [13]:
import pandas as pd

# Load full CSV
df = pd.read_csv("../data/nutrition_filtered.csv", usecols=[0, 1], header=0, names=["dish_id", "calories"])

# Load train/test dish_ids
with open("../data/train_filtered.txt") as f:
    train_ids = set(line.strip() for line in f)

with open("../data/test_filtered.txt") as f:
    test_ids = set(line.strip() for line in f)

#print(df["dish_id"].head(3))
#print(list(train_ids)[:5])

# Filter based on dish_id column (which is named "dish_id")
train_df = df[df["dish_id"].isin(train_ids)].copy()
test_df  = df[df["dish_id"].isin(test_ids)].copy()

# Add image path column
train_df["image_path"] = train_df["dish_id"].apply(lambda x: f"../data/images/{x}.png")
#print(train_df["calories"].head(5))
test_df["image_path"]  = test_df["dish_id"].apply(lambda x: f"../data/images/{x}.png")


In [14]:
import tensorflow as tf

IMG_SIZE = (224, 224)

def preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0  # normalize
    return img, label

def make_dataset(df, batch_size=16, shuffle=True):
    paths = df["image_path"].values
    labels = df["calories"].values.astype("float32")

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(buffer_size=len(df))

    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

# create train and validation datasets
train_ds = make_dataset(train_df, batch_size=16, shuffle=True)
val_ds = make_dataset(test_df, batch_size=16, shuffle=False)

In [15]:
# 1. Imports for model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, Model, Input

# 2. Load pretrained MobileNetV2 (no top, global-avg pooling)
base_model = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)
base_model.trainable = False  # freeze backbone

# 3. Add your regression head
inputs = Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1)(x)  # one scalar output

model = Model(inputs, outputs)

# 4. Compile
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)

# 5. Quick summary
model.summary()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

# 1. Train (no intermediate validation)
history = model.fit(
    train_ds,
    epochs=10
)

# 2. Get predictions + true labels from val_ds
y_true = []
y_pred = []

for batch_imgs, batch_labels in val_ds:
    preds = model(batch_imgs, training=False)        # shape (batch,1)
    y_true.extend(batch_labels.numpy().flatten())   # actual calories
    y_pred.extend(preds.numpy().flatten())          # predicted calories

y_true = np.array(y_true)
y_pred = np.array(y_pred)

# 3. Metrics
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print(f"Final validation MAE: {mae:.2f} kcal")
print(f"Final validation R²: {r2:.3f}")

Epoch 1/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 55s 189ms/step - loss: 68353.5156 - mae: 191.4089
Epoch 2/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 39s 182ms/step - loss: 34334.0352 - mae: 131.5033
Epoch 3/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 38s 174ms/step - loss: 27552.6348 - mae: 118.3314
Epoch 4/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 37s 175ms/step - loss: 26912.4180 - mae: 107.7294
Epoch 5/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 40s 182ms/step - loss: 27242.8242 - mae: 102.8712
Epoch 6/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 38s 177ms/step - loss: 19103.7031 - mae: 95.8012
Epoch 7/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 37s 179ms/step - loss: 28073.7344 - mae: 97.7937
Epoch 8/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 37s 180ms/step - loss: 17699.4883 - mae: 93.9068
Epoch 9/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 38s 183ms/step - loss: 20508.6348 - mae: 93.0958
Epoch 10/10
176/176 ━━━━━━━━━━━━━━━━━━━━ 38s 181ms/step - loss: 18152.7754 - mae: 90.7617
Final validation MAE: 90.28 kcal
Final validation R²: 0.631
